<a href="https://colab.research.google.com/github/MertOzer1/Handwritten-Digit-Recognition-/blob/main/OCR%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/mindee/doctr.git

In [ ]:
!pip install -e doctr/.

In [ ]:
!pip install -r doctr/requirements.txt

In [ ]:
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

In [ ]:
model = ocr_predictor(pretrained=True)
#doc = DocumentFile.from_pdf("2-4 G.pdf").as_images() 
doc = DocumentFile.from_images("1.png")
result = model(doc)
json_output = result.export()

In [ ]:
for w in json_output['pages'][0]['blocks'][0]['lines'][0]['words']:
  print(w['value'])

In [ ]:
import json
with open("out.json","w") as f:
  json.dump(json_output,f)

In [ ]:
print('Number of blocks = ', len(json_output['pages'][0]['blocks']))

In [ ]:
print('Number of lines = ', len(json_output['pages'][0]['blocks'][0]['lines']))

In [ ]:
print('Number of words = ', len(json_output['pages'][0]['blocks'][0]['lines'][0]['words']))

In [ ]:
#words = []
#for w in json_output['pages'][0]['blocks'][0]['lines'][0]['words']:
#  word = (w['value'], w['geometry'][0][0], w['geometry'][0][1])
#  words.append(word)
#  print(w)
for i in range(len(json_output['pages'][0]['blocks'])):
  for j in range(len(json_output['pages'][0]['blocks'][i]["lines"])):
    for w in json_output['pages'][0]['blocks'][i]['lines'][j]['words']:
      if 'SUYUNAMM' in w['value']:
        starting_y = w['geometry'][0][1] # assumes SUYUNAMM comes before all numbers 

seed_names = [str(s) + '.' for s in range(1,28)]
seeds = []
words = []

for i in range(len(json_output['pages'][0]['blocks'])):
  for j in range(len(json_output['pages'][0]['blocks'][i]["lines"])):
    for w in json_output['pages'][0]['blocks'][i]['lines'][j]['words']: 
      if w['geometry'][0][1] <= starting_y:
        continue

      cleanvalue = w["value"].replace("g","9")
      cleanvalue = cleanvalue.replace("i","1")
      cleanvalue = cleanvalue.replace("l","1")
      cleanvalue = cleanvalue.replace("I","1")
      cleanvalue = cleanvalue.replace("ı","1")
      cleanvalue = cleanvalue.replace("s","5")
      cleanvalue = cleanvalue.replace("S","5")
      cleanvalue = cleanvalue.replace("o","0")
      cleanvalue = cleanvalue.replace("O","0")
      cleanvalue = cleanvalue.replace("-",".")
      cleanvalue = cleanvalue.replace("J","1")
      cleanvalue = cleanvalue.replace("/","1")
      cleanvalue = cleanvalue.replace("(","1")
      

      word = (cleanvalue, w['geometry'][0][0], w['geometry'][0][1], w['geometry'][1][1], w['confidence'])
      words.append(word)
      print(w)

      if cleanvalue in seed_names:#  and w['confidence'] > 0.95): #warning 
        seeds.append(word)

In [ ]:
sorted_words = sorted(words, key = lambda x: x[2])
for w in sorted_words:
  print(w)

In [ ]:
def intersect(ymin1, ymax1, ymin2, ymax2):
  return (ymin1 <= ymin2 <= ymax1) or (ymin1 <= ymax2 <= ymax1) or (ymin2 <= ymin1 <= ymax2) or (ymin2 <= ymax1 <= ymax2)

skip = []
lines = []
line = []
for s in seeds:
  line.append(s)
  for w in sorted_words:
    if w not in seeds and w not in skip:
      if intersect(s[2],s[3], w[2],w[3]):
        line.append(w)
        skip.append(w)
      else:
        line.sort(key = lambda x: x[1])
        lines.append(line)
        line = []
        break
for line in lines:
  print(line)

In [ ]:
csv_out = open('output.csv', 'w')
for line in lines:
  for word in line:
    csv_out.write(word[0] + ',')
  csv_out.write('\n')
csv_out.close()

In [ ]:
!pip install xlsxwriter

In [ ]:
from collections import OrderedDict
import xlsxwriter

In [ ]:
# Open an Excel workbook
num=0
while True:
  try:
    file=open("test"+str(num)+".xlsx","r")
    num+=1
  except:
    break;
workbook = xlsxwriter.Workbook('test'+str(num)+'.xlsx')
# Create a sheet
worksheet = workbook.add_worksheet('Data')

In [ ]:
# Set up a format
book_format = workbook.add_format(properties={'bold': True, 'font_color': 'red'})

In [ ]:
# Save the data from the OrderedDict into the excel sheet
for row_num in range(len(lines)):
    for col_num in range(len(lines[row_num])):
        if lines[row_num][col_num][4] < 0.9:
            worksheet.write(row_num, col_num, lines[row_num][col_num][0], book_format)
        else:
            worksheet.write(row_num, col_num, lines[row_num][col_num][0])

# Close the workbook
workbook.close()